In [1]:
import os
import pandas as pd
import zipfile

In [2]:
# Seteo la ruta del repositorio (reemplazarla por la propia)
ruta = r"C:\Users\Fran\MAESTRÍA\Machine Learning\Tarea 2"

In [3]:
# Fijo rutas para cada carpeta del repositorio
document_path = os.path.join(ruta, "document")
scripts_path = os.path.join(ruta, "scripts")
stores_path = os.path.join(ruta, "stores")
views_path = os.path.join(ruta, "views")

zip_path = os.path.join(stores_path, "mlunlp-2024-ps-2.zip")


In [4]:
train_hogares_path = "train_hogares.csv"
train_personas_path = "train_personas.csv"
test_hogares_path = "test_hogares.csv"
test_personas_path = "test_personas.csv"

In [9]:
with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open(train_hogares_path) as f:
        train_hogares = pd.read_csv(f)

with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open(train_personas_path) as f:
        train_personas = pd.read_csv(f)

with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open(test_hogares_path) as f:
        test_hogares = pd.read_csv(f)

with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open(test_personas_path) as f:
        test_personas = pd.read_csv(f)

In [10]:
train_merged = pd.merge(train_personas, train_hogares, on='id', how= "inner", suffixes=('_ind', '_hogar'))  
test_merged = pd.merge(test_personas, test_hogares, on='id', how= "inner", suffixes=('_ind', '_hogar'))  

**Creación y transformación de variables**

Cuestiones a tener en cuenta: 

Se cambió el formato de las variables "P6210", "P6100", "P6585s3", "P6585s1" de float a entero.
Hacerlo también para las bases de testeo

In [11]:
columnas = train_merged.columns.tolist()
print(columnas)

columnas2 = test_merged.columns.tolist()
print(columnas2)

['id', 'Orden', 'Clase_ind', 'Dominio_ind', 'Estrato1', 'P6020', 'P6040', 'P6050', 'P6090', 'P6100', 'P6210', 'P6210s1', 'P6240', 'Oficio', 'P6426', 'P6430', 'P6500', 'P6510', 'P6510s1', 'P6510s2', 'P6545', 'P6545s1', 'P6545s2', 'P6580', 'P6580s1', 'P6580s2', 'P6585s1', 'P6585s1a1', 'P6585s1a2', 'P6585s2', 'P6585s2a1', 'P6585s2a2', 'P6585s3', 'P6585s3a1', 'P6585s3a2', 'P6585s4', 'P6585s4a1', 'P6585s4a2', 'P6590', 'P6590s1', 'P6600', 'P6600s1', 'P6610', 'P6610s1', 'P6620', 'P6620s1', 'P6630s1', 'P6630s1a1', 'P6630s2', 'P6630s2a1', 'P6630s3', 'P6630s3a1', 'P6630s4', 'P6630s4a1', 'P6630s6', 'P6630s6a1', 'P6750', 'P6760', 'P550', 'P6800', 'P6870', 'P6920', 'P7040', 'P7045', 'P7050', 'P7070', 'P7090', 'P7110', 'P7120', 'P7140s1', 'P7140s2', 'P7150', 'P7160', 'P7310', 'P7350', 'P7422', 'P7422s1', 'P7472', 'P7472s1', 'P7495', 'P7500s1', 'P7500s1a1', 'P7500s2', 'P7500s2a1', 'P7500s3', 'P7500s3a1', 'P7505', 'P7510s1', 'P7510s1a1', 'P7510s2', 'P7510s2a1', 'P7510s3', 'P7510s3a1', 'P7510s5', 'P751

In [12]:
# Creo la variable hacin
train_merged["hacin"] = train_merged["Nper"] / train_merged["P5010"]
test_merged["hacin"] = test_merged["Nper"] / test_merged["P5010"]

In [13]:
# Transformo los valores de P6210
train_merged["P6210"] = train_merged["P6210"].fillna(0).astype(int) 
print(train_merged["P6210"].unique())

test_merged["P6210"] = train_merged["P6210"].fillna(0).astype(int) 

[3 2 5 4 1 6 0 9]


In [16]:
# Obtengo el máximo nivel educativo por hogar
educacion_max_hogar = train_merged.groupby("id")["P6210"].max().reset_index()
educacion_max_hogar2 = test_merged.groupby("id")["P6210"].max().reset_index()

# Renombro la columna
educacion_max_hogar.rename(columns={"P6210": "educ_max"}, inplace=True)
educacion_max_hogar2.rename(columns={"P6210": "educ_max"}, inplace=True)

train_merged = train_merged.merge(educacion_max_hogar, on="id", how="left")
test_merged = test_merged.merge(educacion_max_hogar2, on="id", how="left")

In [17]:
# Transformo los valores de P6100
train_merged["P6100"] = train_merged["P6100"].fillna(0).astype(int) 
test_merged["P6100"] = test_merged["P6100"].fillna(0).astype(int) 

# Creo dummy con personas de régimen de salud subsidiado
test_merged["salud_subsi"] = (test_merged["P6100"] == 3).astype(int)
train_merged["salud_subsi"] = (train_merged["P6100"] == 3).astype(int)

In [18]:
# Transformo los valores de P6585s3
train_merged["P6585s3"] = train_merged["P6585s3"].fillna(0).astype(int) 
test_merged["P6585s3"] = test_merged["P6585s3"].fillna(0).astype(int) 

# Asigno valor 1 a quienes reportan que sí y 0 en otro caso
train_merged["fam_subsi"] = (train_merged["P6585s3"] == 1).astype(int)

test_merged["fam_subsi"] = (test_merged["P6585s3"] == 1).astype(int)

In [19]:
# Transformo los valores de P6585s1
train_merged["P6585s1"] = train_merged["P6585s1"].fillna(0).astype(int) 
test_merged["P6585s1"] = test_merged["P6585s1"].fillna(0).astype(int) 

# Asigno valor 1 a quienes reportan que sí y 0 en otro caso
train_merged["alim_subsi"] = (train_merged["P6585s1"] == 1).astype(int)
test_merged["alim_subsi"] = (test_merged["P6585s1"] == 1).astype(int)

In [20]:
# Transformo los valores de P6585s2
train_merged["P6585s2"] = train_merged["P6585s2"].fillna(0).astype(int) 
test_merged["P6585s2"] = test_merged["P6585s2"].fillna(0).astype(int) 

# Asigno valor 1 a quienes reportan que sí y 0 en otro caso
train_merged["transp_subsi"] = (train_merged["P6585s2"] == 1).astype(int)
test_merged["transp_subsi"] = (test_merged["P6585s2"] == 1).astype(int)

In [21]:
# Selecciono las variables que podría llegar a utilizar para estimar pobreza con el primer approach
train_pobreza = train_merged[["id", "Orden", "P6040", "hacin", "Clase_hogar", "educ_max", "salud_subsi", "fam_subsi", "alim_subsi", "transp_subsi", "P6430", "Lp", "Pobre"]]
test_pobreza = test_merged[["id", "Orden", "P6040", "hacin", "Clase_hogar", "educ_max", "salud_subsi", "fam_subsi", "alim_subsi", "transp_subsi", "P6430", "Lp"]]

In [22]:
# Selecciono las variables que podría llegar a utilizar para estimar pobreza con el segundo approach

train_ingreso = train_merged[["id", "Orden", "Ingtot", "P6020", "P6040", "P6020", "P6920", "P6800", "P6210", "P6426", "P6430","educ_max", "Lp"]]
test_ingreso = test_merged[["id", "Orden", "P6020", "P6040", "P6020", "P6920", "P6800", "P6210", "P6426", "P6430","educ_max", "Lp"]]

# "P6040": edad, 
# "P6020": sexo, 
# "P6920": cotiza pension, 
# "P6800": horas de trabajo semanal normal, 
# "P6210": máximo nivel educativo, 
# "P6426": antigüedad, 
# "P6920": tipo de ocupación

In [23]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report


# Preprocesamiento de los datos
# Selección de variables relevantes y eliminación de valores faltantes
data_train = train_pobreza[["id", "P6040", "hacin", "educ_max", "salud_subsi", "fam_subsi", "P6430", "Lp", "Pobre"]].dropna()
data_test = test_pobreza[["id", "P6040", "hacin", "educ_max", "salud_subsi", "fam_subsi", "P6430", "Lp"]].dropna()

# Crear un dataset a nivel de hogar agrupando por id
hogares_train = data_train.groupby('id').mean()
hogares_train['Pobre'] = data_train.groupby('id')['Pobre'].max()  # Si algún miembro es pobre, se clasifica el hogar como pobre
hogares_test = data_test.groupby('id').mean()

# Definir variables predictoras y objetivo para entrenamiento
X_train = hogares_train.drop(columns=['Pobre'])
y_train = hogares_train['Pobre']

# Definir variables predictoras para prueba
X_test = hogares_test

# Modelos
modelos = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Elastic Net': ElasticNet(alpha=0.1, l1_ratio=0.5),
    'Decision Tree': DecisionTreeClassifier(),
    'Decision Tree (max_depth=10)': DecisionTreeClassifier(max_depth=10),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Random Forest (n_estimators=200, max_depth=10)': RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42),
    'Gradient Boosting (n_estimators=200, learning_rate=0.05)': GradientBoostingClassifier(n_estimators=200, learning_rate=0.05, random_state=42)
}

# Entrenamiento y evaluación
resultados = {}

for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_train)  # Evaluación en datos de entrenamiento
    if nombre == 'Elastic Net':
        y_pred = (y_pred > 0.5).astype(int)  # Convertir predicciones en binarias para Elastic Net
    accuracy = accuracy_score(y_train, y_pred)
    resultados[nombre] = accuracy
    print(f"Modelo: {nombre}")
    print(f"Exactitud en entrenamiento: {accuracy:.2f}")
    print(classification_report(y_train, y_pred))
    print("-" * 40)

# Comparar resultados
print("Resultados finales:")
for modelo, accuracy in resultados.items():
    print(f"{modelo}: {accuracy:.2f}")

# Seleccionar el mejor modelo
best_model_name = max(resultados, key=resultados.get)
best_model = modelos[best_model_name]

# Crear pipeline final
final_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', best_model)
])

# Entrenar pipeline final
final_pipeline.fit(X_train, y_train)

# Predicciones finales
hogares_test['Predicted_Pobre'] = final_pipeline.predict(X_test)

# Calcular proporción de hogares pobres en el conjunto de prueba
prop_pobres = hogares_test['Predicted_Pobre'].mean()
print(f"Proporción de hogares pobres predicha: {prop_pobres:.2%}")


Modelo: Logistic Regression
Exactitud en entrenamiento: 0.83
              precision    recall  f1-score   support

           0       0.85      0.96      0.90    116213
           1       0.59      0.26      0.36     26524

    accuracy                           0.83    142737
   macro avg       0.72      0.61      0.63    142737
weighted avg       0.80      0.83      0.80    142737

----------------------------------------
Modelo: Elastic Net
Exactitud en entrenamiento: 0.82
              precision    recall  f1-score   support

           0       0.82      1.00      0.90    116213
           1       0.77      0.03      0.05     26524

    accuracy                           0.82    142737
   macro avg       0.79      0.51      0.47    142737
weighted avg       0.81      0.82      0.74    142737

----------------------------------------
Modelo: Decision Tree
Exactitud en entrenamiento: 0.99
              precision    recall  f1-score   support

           0       0.99      1.00      1

In [24]:
resultados

{'Logistic Regression': 0.8291262952142752,
 'Elastic Net': 0.8175665734883036,
 'Decision Tree': 0.9928189607459874,
 'Decision Tree (max_depth=10)': 0.8526310627237507,
 'Random Forest': 0.9927629136103463,
 'Random Forest (n_estimators=200, max_depth=10)': 0.8506904306521785,
 'Gradient Boosting': 0.8475798146240988,
 'Gradient Boosting (n_estimators=200, learning_rate=0.05)': 0.847691908895381}